<a href="https://colab.research.google.com/github/hrtechplus/sample/blob/main/training_the_spacy_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install spacy==3.5.3


In [2]:
pip install --upgrade spacy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 91.2 MB/s eta 0:00:00
  Attempting uninstall: blis
    Found existing installation: blis 0.7.11
    Uninstalling blis-0.7.11:
      Successfully uninstalled blis-0.7.11
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.12
    Uninstalling thinc-8.1.12:
      Successfully uninstalled thinc-8.1.12
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.3
    Uninstalling spacy-3.5.3:
      Successfully uninstalled spacy-3.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.8.4 which is incompatible.


In [1]:
pip install spacy-lookups-data


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 6.4 MB/s eta 0:00:00


In [33]:
!python -m spacy validate

2025-01-26 08:44:03.883880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-26 08:44:03.917282: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-26 08:44:03.927454: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-26 08:44:05.650550: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.5.3) =================
ℹ spaCy installation: /usr/local/lib/python3.11/dist-packages/spacy

NAME             SPACY            VERSION                      

In [3]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 47.7 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.5.0
    Uninstalling en-core-web-sm-3.5.0:
      Successfully uninstalled en-core-web-sm-3.5.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Working Data set training  - only single form

In [3]:

import spacy
from spacy.training.example import Example
import random
import json

# Load a blank English model with a config
nlp = spacy.blank("en")
nlp.add_pipe("ner")

# Add the new labels to the NER pipe
labels = ["ORDER_NUMBER", "CUSTOMER_NAME", "STATUS", "TOTAL_AMOUNT", "CATEGORY", "PRODUCT_NAME", "STOCK_LEVEL"]
ner = nlp.get_pipe("ner")  # Get the NER pipe
for label in labels:
    ner.add_label(label)

# Load your training data
with open("updated_training_data_format.json", "r") as f:
    TRAIN_DATA = json.load(f)

# Prepare the training data for spaCy
train_examples = []
for entry in TRAIN_DATA:
    text = entry[0]  # Extract text
    annotations = entry[1]  # Extract annotations
    doc = nlp.make_doc(text)  # Create a Doc object
    try:
        example = Example.from_dict(doc, annotations)
        train_examples.append(example)
    except ValueError as e:
        print(f"Skipping example: {text} due to error: {e}")

# Start the training loop
nlp.initialize()  # Initialize the pipeline before training

# Loop through the training data and update the model
for epoch in range(30):  # Number of epochs, adjust as needed
    random.shuffle(train_examples)
    losses = {}
    # Batch the examples and update the model
    for batch in spacy.util.minibatch(train_examples, size=8):  # Adjust batch size as needed
        nlp.update(batch, drop=0.5, losses=losses)  # Dropout for regularization
    print(f"Epoch {epoch}, Losses: {losses}")
# Save the trained model
nlp.to_disk("trained_spacy_model_with_all_entities")



Epoch 0, Losses: {'ner': 1278.2614458171229}
Epoch 1, Losses: {'ner': 283.03383382268163}
Epoch 2, Losses: {'ner': 55.760036261085645}
Epoch 3, Losses: {'ner': 35.371331604682915}
Epoch 4, Losses: {'ner': 14.626114862200009}
Epoch 5, Losses: {'ner': 24.210669768905646}
Epoch 6, Losses: {'ner': 12.325913082509361}
Epoch 7, Losses: {'ner': 20.452155241916458}
Epoch 8, Losses: {'ner': 11.137794055379064}
Epoch 9, Losses: {'ner': 3.548137742198718}
Epoch 10, Losses: {'ner': 9.534345697555217}
Epoch 11, Losses: {'ner': 8.386024800372333}
Epoch 12, Losses: {'ner': 2.4396776015094175}
Epoch 13, Losses: {'ner': 16.41818087916556}
Epoch 14, Losses: {'ner': 16.631243453082917}
Epoch 15, Losses: {'ner': 11.690356485983129}
Epoch 16, Losses: {'ner': 9.711229036808605}
Epoch 17, Losses: {'ner': 6.085731740214576}
Epoch 18, Losses: {'ner': 5.498750895367103}
Epoch 19, Losses: {'ner': 6.24772724135789}
Epoch 20, Losses: {'ner': 25.810767813359682}
Epoch 21, Losses: {'ner': 14.621841438121818}
Epoch 2

**Working Test Data**

In [6]:
# Test the model (optional)
test_text = " John Doe Pending 250.00 Electronics Galaxy S25 Ultra 50 ORD12345"
doc = nlp(test_text)
print(f"Input Text: {test_text}")
print("-" * 30)  # Separator for better visual distinction
for ent in doc.ents:
    print("{:<20} {:<15}".format(ent.text, ent.label_))
print("-" * 30)

Input Text:  John Doe Pending 250.00 Electronics Galaxy S25 Ultra 50 ORD12345
------------------------------
John Doe             CUSTOMER_NAME  
Pending              STATUS         
250.00 Electronics   CUSTOMER_NAME  
Galaxy S25 Ultra     PRODUCT_NAME   
50                   STOCK_LEVEL    
ORD12345             ORDER_NUMBER   
------------------------------


In [23]:
import spacy
import random
import json
from spacy.training.example import Example
from spacy.util import minibatch

# Load a blank English model with a config
nlp = spacy.blank("en")

# Add the NER pipeline
ner = nlp.add_pipe("ner")

# Add the new labels to the NER pipe
labels = ["ORDER_NUMBER", "CUSTOMER_NAME", "STATUS", "TOTAL_AMOUNT", "CATEGORY", "PRODUCT_NAME", "STOCK_LEVEL"]
for label in labels:
    ner.add_label(label)

# Load your training data
with open("newFormat.json", "r") as f:
    TRAIN_DATA = json.load(f)

# Prepare the training data for spaCy
train_examples = []
# Iterate through the training data, assuming it's a list of lists
for entry in TRAIN_DATA:
    # Access data by index if it is a list of lists, example:
    text = entry[0]  # Text is assumed to be the first element
    annotations = entry[1]  # Annotations are assumed to be the second element

    # If your data structure is different, adjust the indices accordingly
    # For example, if 'text' and 'annotations' are keys in a dictionary within the list,
    # you would use:
    # text = entry['text']
    # annotations = entry['annotations']

    doc = nlp.make_doc(text)  # Create a Doc object
    try:
        example = Example.from_dict(doc, annotations)
        train_examples.append(example)
    except ValueError as e:
        print(f"Skipping example: {text} due to error: {e}")

# Step 1: Initialize the pipeline before training
nlp.initialize()

# Step 2: Start the training loop (30 epochs)
for epoch in range(30):
    print(f"Epoch {epoch + 1}")
    random.shuffle(train_examples)  # Shuffle the training data
    losses = {}

    # Step 3: Batch the examples and update the model
    for batch in minibatch(train_examples, size=8):  # Adjust batch size as needed
        nlp.update(batch, drop=0.5, losses=losses)  # Dropout for regularization

    print(f"Losses: {losses}")

# Step 4: Save the trained model
nlp.to_disk("trained_spacy_model_with_all_entities2")
print("Model trained and saved successfully!")

Epoch 1
Losses: {'ner': 1391.2725388257002}
Epoch 2
Losses: {'ner': 373.2854748728338}
Epoch 3
Losses: {'ner': 90.74356207398945}
Epoch 4
Losses: {'ner': 43.92244278994606}
Epoch 5
Losses: {'ner': 28.706541877106254}
Epoch 6
Losses: {'ner': 21.34786547955789}
Epoch 7
Losses: {'ner': 27.606747138059216}
Epoch 8
Losses: {'ner': 22.3009180780196}
Epoch 9
Losses: {'ner': 10.805384086360558}
Epoch 10
Losses: {'ner': 21.162404676905332}
Epoch 11
Losses: {'ner': 15.324067428359406}
Epoch 12
Losses: {'ner': 9.671038264174083}
Epoch 13
Losses: {'ner': 16.89879074162068}
Epoch 14
Losses: {'ner': 25.578093557469135}
Epoch 15
Losses: {'ner': 26.348526723489467}
Epoch 16
Losses: {'ner': 14.083555447958847}
Epoch 17
Losses: {'ner': 20.908121236687652}
Epoch 18
Losses: {'ner': 28.92465559709868}
Epoch 19
Losses: {'ner': 22.409981583203784}
Epoch 20
Losses: {'ner': 9.277391248251057}
Epoch 21
Losses: {'ner': 13.51852037892904}
Epoch 22
Losses: {'ner': 19.55675017315914}
Epoch 23
Losses: {'ner': 17.380

In [24]:
import spacy

# Load the fine-tuned model
nlp = spacy.load("trained_spacy_model_with_all_entities2")

# Test input queries
test_queries = [
    "What is the status of order ORD10001?",
    "Show me all orders for customer Bob Brown.",
    "What is the stock for the MacBook Pro?",
    "How many units of Galaxy S25 Ultra in stock?",
    "Can you show me all orders for customer Alice Johnson with product PS5?"

]

# Process each test query
for query in test_queries:
    doc = nlp(query)  # Process the input with the trained model

    print(f"Input: {query}")
    for ent in doc.ents:  # Print the recognized entities
        print(f"Entity: {ent.text}, Label: {ent.label_}")
    print("-" * 50)  # Separator for clarity between results




Input: What is the status of order ORD10001?
--------------------------------------------------
Input: Show me all orders for customer Bob Brown.
Entity: Bob Brown, Label: CUSTOMER_NAME
--------------------------------------------------
Input: What is the stock for the MacBook Pro?
Entity: MacBook Pro, Label: PRODUCT_NAME
--------------------------------------------------
Input: How many units of Galaxy S25 Ultra in stock?
Entity: Galaxy S25 Ultra, Label: PRODUCT_NAME
--------------------------------------------------
Input: Can you show me all orders for customer Alice Johnson with product PS5?
Entity: Alice Johnson, Label: CUSTOMER_NAME
Entity: PS5?, Label: ORDER_NUMBER
--------------------------------------------------


# **Combined the model**

In [2]:
import spacy

# Load the fine-tuned model
nlp = spacy.load("fine_tuned_order_model")  # Ensure this matches the path where your model is saved

# Test input queries
test_inputs = [
    "What is the status of order ORD10001?",
    "Show me all orders for customer Bob Brown.",
    "List all products in the Gaming category.",
    "How many Galaxy S25 in the stock?",
    "How many units of iPhone 12 in stock?",
    "What orders were placed in the last month?",
    "Can you show all orders for customer Bob Brown with the product Galaxy S25?",
    "How many Samsung TVs and Apple Watches do we have in stock?",
    "How many Electronics products are in stock?"
]

# Process each test input and extract recognized entities
for test_input in test_inputs:
    doc = nlp(test_input)  # Process the input with the fine-tuned model

    print(f"Input: {test_input}")

    # Print out the recognized entities
    for ent in doc.ents:
        print(f"Entity: {ent.text}, Label: {ent.label_}")

    # Add a separator for clarity
    print("-" * 50)  # Separator between results


Input: What is the status of order ORD10001?
--------------------------------------------------
Input: Show me all orders for customer Bob Brown.
--------------------------------------------------
Input: List all products in the Gaming category.
--------------------------------------------------
Input: How many Galaxy S25 in the stock?
--------------------------------------------------
Input: How many units of iPhone 12 in stock?
--------------------------------------------------
Input: What orders were placed in the last month?
--------------------------------------------------
Input: Can you show all orders for customer Bob Brown with the product Galaxy S25?
--------------------------------------------------
Input: How many Samsung TVs and Apple Watches do we have in stock?
--------------------------------------------------
Input: How many Electronics products are in stock?
--------------------------------------------------
